In [ ]:
# Imports
from google.cloud import bigquery
from google.cloud import storage
from unidecode import unidecode
import pandas as pd
import numpy as np

def valid_event(event, context):
    '''
        Valida que el evento corresponda antes de
        ejecutar el resto de la función main
    '''
    if context.event_type != "google.storage.object.finalize":
        print("Evento no es subida de archivo finalizada. Fin de ejecución")
        return False

    if event["name"][-4:] != ".csv":
        print("Archivo subido no es archivo .csv ( extensión .csv)")
        return False

    return True

def instantiate_api_clients():
    # Creamos cliente de api de storage autenticado
    storage_client = None
    storage_client = storage.Client()

    # Creamos cliente de api de BigQuery autenticado
    bigquery_client = None
    bigquery_client = bigquery.Client()

    return storage_client, bigquery_client

def clean_data(dataframe):
    try:

        columns_to_drop = ['Rut Fijo', 'CS',"SchamanDirecto","SchamanIVR","Voz","Intermitencia","Lentitud","Luces Rojas","Luces Apagadas","Luces Normales","Error Diagnostico","Sin Energia","Agendamiento","Ticket N2","C2C_N1","Feedback","Rut Administrador"]
        # Eliminar otras columnas y mantener solo las necesarias para el flujo
        df1 = dataframe.drop(columns=columns_to_drop)

        # Proceso de cambiar valores superiores a 1 por 1
        df1[df1.columns[12:]] = df1[df1.columns[12:]].apply(pd.to_numeric, errors='coerce')

        # Apply the clip method
        df1[df1.columns[12:]] = df1[df1.columns[12:]].clip(upper=1)

        # Listado de autoatenciones
        values_to_check = ['Facturacion', 'Consultar_Trafico_Nacional', 'Teardown',"Informacion_Plan","Consultar_Trafico","Obtener_Contraseña"]

        # Añadir columna 'Autoatencion' a partir del listado
        df1['Autoatencion'] = df1['intent'].apply(lambda x: 1 if isinstance(x, str) and any(value in x for value in values_to_check) else 0)

        # Columna transferencia plus autoatencion
        columns_to_sum2 = ['Transferidas a Ejecutivo', 'Autoatencion']
        df1['Transferencia_auto'] = (df1[columns_to_sum2].sum(axis=1) >= 2).astype(int)

        # Eliminar otras columnas y mantener solo las necesarias para el flujo
        df1= df1[["value.context.conversation_id","ANI","Fecha","Segmento","Rut Empresa","Tipo de Cliente", "EPA","Resolucion","Transferidas a Ejecutivo", "intent","Autoatencion","Transferencia_auto","error_api","FullStackB2B"]]
        df1 = df1.rename(columns={'value.context.conversation_id': 'Conversation_id', 'Rut Empresa': 'Rut_empresa', 'Tipo de Cliente': 'Tipo_de_cliente', "Transferidas a Ejecutivo": "Transferidas_a_ejecutivo", "intent":"Event"}).copy()

        # Quedarnos con el ultimo registro de EPA (nota) dentro de una misma fila y ajuste en el formato del campo
        df1.loc[:, "EPA"] = df1["EPA"].replace({5.0: "5", 3.0: "3", 4.0: "4", 1.0: "1", 2.0: "2"})
        df1.loc[:, 'EPA'] = df1['EPA'].str.split().str[-1]

        # Quedarnos con el ultimo registro de resolucion (si o no) dentro de una misma fila
        df1.loc[:, 'Resolucion'] = df1['Resolucion'].str.split().str[-1]

        # Quedarnos con el ultimo registro de Fecha dentro de una misma fila y poder leer la columna correctamente como fecha
        df1.loc[:, 'Fecha'] = df1['Fecha'].str.split().str[-1]

        # Parseo de conversatio_id como string
        df1['Conversation_id'] = df1['Conversation_id'].astype('string')

        # Cambiar todos los numeros superiores a 1 por un 1
        df1.loc[:, 'Transferidas_a_ejecutivo'] = df1['Transferidas_a_ejecutivo'].clip(upper=1)

        # Eliminar espacios vacios de la columna
        # Convertir la columna a datetime dtype
        df1['Fecha'] = pd.to_datetime(df1['Fecha'].astype(str).str.strip(), dayfirst=True)

        # Función para remover acentos y ñ
        def remove_accents(text):
            if pd.notna(text):
                return unidecode(str(text))
            else:
                return text

        # Remover acentos en las columnas de segmentos y resolucion
        df1.loc[:, 'Segmento'] = df1['Segmento'].apply(remove_accents)
        df1.loc[:, 'Resolucion'] = df1['Resolucion'].apply(remove_accents)

        # Cambiar Si por 1 y No por 0
        df1['Resolucion'] = df1['Resolucion'].replace({'Si': 1, 'No': 0}).astype(np.float64)

        # Convertir los nan como texto vacio
        df1.loc[:, "Conversation_id"] = df1['Conversation_id'].fillna("")
        df1.loc[:, 'ANI'] = df1['ANI'].fillna("")
        df1.loc[:, 'Fecha'] = df1['Fecha'].fillna("")
        df1.loc[:, 'Segmento'] = df1['Segmento'].fillna("")
        df1.loc[:, "Rut_empresa"] = df1['Rut_empresa'].fillna("")
        df1.loc[:, 'Tipo_de_cliente'] = df1['Tipo_de_cliente'].fillna("")
        df1.loc[:, 'EPA'] = df1['EPA'].fillna("")
        df1.loc[:, 'Resolucion'] = df1['Resolucion'].fillna("")
        df1.loc[:, 'Transferidas_a_ejecutivo'] = df1['Transferidas_a_ejecutivo'].fillna("")
        df1.loc[:, 'Event'] = df1['Event'].fillna("")
        df1.loc[:, 'Autoatencion'] = df1['Autoatencion'].fillna("")
        df1.loc[:, 'Transferencia_auto'] = df1['Transferencia_auto'].fillna("")
        df1.loc[:, 'error_api'] = df1['error_api'].fillna("")
        df1.loc[:, 'FullStackB2B'] = df1['FullStackB2B'].fillna("")

        # Setear tipo de datos como string para evitar conflictos en la subida
        df1['Resolucion'] = df1['Resolucion'].astype("string")
        df1['ANI'] = df1['ANI'].astype("string")
        df1['Segmento'] = df1['Segmento'].astype("string")
        df1['Rut_empresa'] = df1['Rut_empresa'].astype("string")
        df1['Tipo_de_cliente'] = df1['Tipo_de_cliente'].astype("string")
        df1['EPA'] = df1['EPA'].astype("string")
        df1['Transferidas_a_ejecutivo'] = df1['Transferidas_a_ejecutivo'].astype("string")
        df1['Tipo_de_cliente'] = df1['Tipo_de_cliente'].astype("string")
        df1['EPA'] = df1['EPA'].astype("string")
        df1['Transferidas_a_ejecutivo'] = df1['Transferidas_a_ejecutivo'].astype("string")
        df1['Event'] = df1['Event'].astype("string")
        df1['Autoatencion'] = df1['Autoatencion'].astype("string")
        df1['Transferencia_auto'] = df1['Transferencia_auto'].astype("string")
        df1['error_api'] = df1['error_api'].astype("string")
        df1['FullStackB2B'] = df1['FullStackB2B'].astype("string")

        return df1, True

    except Exception as err:
        print('Error limpiando la data')
        print(err)
        return None, False